# 作業：基因演算法

- 程式檔
- 影片檔（文字詳細說明也可）
- 兩張圖
    - log: Gen, best fit, x, y, x+y, history_best_fit
    - 圖表：Fitness/lteration

## 題目

1. **目標函數**：

   $$\text{Minimize } f(x, y) = -\cos(\pi y) - \exp(-\pi (x - 0.5)^2) \sin^2(\pi x)$$

2. **約束條件**：

   $$-1 \leq x \leq 2$$
   $$4 \leq y \leq 6$$
   $$x + y \geq 5$$


2. **答案**：

   $$\text{Minimum } = -2 \text{ at } (x, y) = (0.5, 6)$$


# 程式碼

In [ ]:
from abc import ABC, abstractmethod
import numpy as np
from typing import List

### 演算法參數設定

在 Jupyter Notebook 中，程式碼儲存格是依序執行的，每個儲存格的執行結果會儲存在記憶體中。

如果你修改了一個儲存格的內容（例如修改了變數的值），但沒有重新執行與其相關的所有儲存格，那麼這些修改不會影響到已經執行過的程式碼。

In [ ]:
class Config:
    POP_SIZE = 500  # 種群大小
    GENERATIONS = 100  # 世代數
    MUTATION_DELTA = 0.5  # 變異幅度
    MUTATION_RATE = 0.6  # 變異率

    SELECTION_STRATEGY = "tournament"  # "roulette", "tournament", "rank_based"
    TOURNAMENT_SIZE = 5  # 錦標賽選擇(tournament)的大小

### 目標函數

$$\text{Minimize } f(x, y) = -\cos(\pi y) - \exp(-\pi (x - 0.5)^2) \sin^2(\pi x)$$

In [ ]:
class ObjectiveFunction:
    @staticmethod
    def evaluate(x, y):
        return -np.cos(np.pi * y) - np.exp(-np.pi * (x - 0.5)**2) * np.sin(np.pi * x)**2

### 個體基因

兩個基因，分別代表 x, y

   $$-1 \leq x \leq 2$$
   $$4 \leq y \leq 6$$
   $$x + y \geq 5$$

In [ ]:
class Individual:
    # 初始化個體，沒有提供 x 和 y 則隨機生成
    def __init__(self, x=None, y=None):
        if x is None or y is None:
            self.x = np.random.uniform(-1, 2)
            self.y = np.random.uniform(4, 6)
            while self.x + self.y < 5:
                self.x = np.random.uniform(-1, 2)
                self.y = np.random.uniform(4, 6)
        else:
            self.x = x
            self.y = y

    # 計算個體的適應度
    def fitness(self):
        return ObjectiveFunction.evaluate(self.x, self.y)

    # 根據變異率對個體進行變異
    def mutate(self, mutation_rate=Config.MUTATION_RATE):
        if np.random.rand() < mutation_rate: # 以 mutation_rate 的機率進行變異
            self.x += np.random.uniform(-Config.MUTATION_DELTA, Config.MUTATION_DELTA) # 變異幅度
            self.y += np.random.uniform(-Config.MUTATION_DELTA, Config.MUTATION_DELTA)
            self.x = np.clip(self.x, -1, 2) # 防止變異後超出範圍，限制在 [-1, 2] 和 [4, 6] 之間
            self.y = np.clip(self.y, 4, 6)
            if self.x + self.y < 5: # 如果變異後不符合限制條件，
                self.y = 5 - self.x # ???

### 選擇方法

基因演算法（Genetic Algorithm, GA）中，選擇策略是決定哪些個體（解）會被選擇來產生下一代。這些策略的目的是確保好的個體有更高的機會被選中，同時維持一定的多樣性。

1. **輪盤賭選擇（Roulette Wheel Selection）**：
    - **原理**：這種方法根據個體的適應度來選擇個體。適應度越高的個體被選中的概率越大。可以想像成一個輪盤，每個個體占據的區域大小與其適應度成正比。
    - **優點**：簡單直觀，適應度高的個體有較大機會被選中。
    - **缺點**：如果適應度差異很大，適應度低的個體幾乎沒有機會被選中，可能導致早熟收斂（premature convergence）。

2. **錦標賽選擇（Tournament Selection）**：
    - **原理**：從種群中隨機選擇一小部分個體（通常稱為錦標賽大小），然後從中選擇適應度最高的個體。這個過程重複多次，直到選出足夠多的個體。
    - **優點**：實現簡單，能夠調整選擇壓力（通過改變錦標賽大小來控制選擇壓力），適應度差異不大的情況下效果較好。
    - **缺點**：如果錦標賽大小過大，可能導致選擇壓力過大，從而導致多樣性降低；如果過小，選擇壓力過小，進化速度變慢。

3. **基於排名的選擇（Rank-Based Selection）**：
    - **原理**：首先根據個體的適應度對整個種群進行排序，然後根據個體的排名來分配選擇概率。排名越高的個體選擇概率越大，但這種概率分配是基於排名而不是適應度。
    - **優點**：能夠避免適應度值差異過大導致的問題，保持種群多樣性，防止早熟收斂。
    - **缺點**：需要對種群進行排序，可能會增加計算複雜度。

總結：
- **輪盤賭選擇**適合適應度分佈較均勻的情況，但可能會導致早熟收斂。
- **錦標賽選擇**靈活性較高，可以通過調整錦標賽大小來控制選擇壓力，適應範圍較廣。
- **基於排名的選擇**能夠有效避免適應度差異過大的問題，保持種群多樣性，但計算量相對較大。

In [ ]:
# from abc import ABC, abstractmethod
# import numpy as np
# from typing import List

class SelectionStrategy(ABC):
    @abstractmethod
    def select_parents(self, population: List[Individual]) -> List[Individual]:
        pass

class RouletteWheelSelection(SelectionStrategy):
    def select_parents(self, population: List[Individual]) -> List[Individual]:
        fitness = np.array([individual.fitness() for individual in population])
        fitness -= fitness.min()
        if fitness.sum() == 0:
            fitness = np.ones_like(fitness)
        probabilities = fitness / fitness.sum()
        selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
        return [population[i] for i in selected_indices]

class TournamentSelection(SelectionStrategy):
    def __init__(self, tournament_size: int):
        self.tournament_size = tournament_size

    def select_parents(self, population: List[Individual]) -> List[Individual]:
        fitness = np.array([individual.fitness() for individual in population])
        selected = []
        for _ in range(len(population)):
            tournament = np.random.choice(len(population), self.tournament_size)
            best = tournament[np.argmax(fitness[tournament])]
            selected.append(population[best])
        return selected

class RankBasedSelection(SelectionStrategy):
    def select_parents(self, population: List[Individual]) -> List[Individual]:
        fitness = np.array([individual.fitness() for individual in population])
        ranks = np.argsort(np.argsort(fitness))
        probabilities = ranks / ranks.sum()
        selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
        return [population[i] for i in selected_indices]

### 交配並產生下一代

**Selection：**
- 使用上面三種選擇方法

**Crossover：**
- 線性插值交叉（Linear Interpolation Crossover）
    - 通過在父母個體之間進行線性插值來生成子代個體，屬於連續變量優化問題中的交叉技術。

**Mutation：**
- 位於 Individual class 中。
- 均勻突變（Uniform Mutation）

In [ ]:
class Population:
    def __init__(self, size, selection_strategy: SelectionStrategy):
        self.individuals = [Individual() for _ in range(size)]
        self.selection_strategy = selection_strategy

    # 計算種群中所有個體的適應度
    def evaluate(self):
        return [individual.fitness() for individual in self.individuals]

    # 選擇父母
    def select_parents(self):
        return self.selection_strategy.select_parents(self.individuals) # 由選擇策略來選擇父母

    def crossover(self, parent1, parent2):
        alpha = np.random.rand()
        x_child = alpha * parent1.x + (1 - alpha) * parent2.x
        y_child = alpha * parent1.y + (1 - alpha) * parent2.y
        return Individual(x_child, y_child)

    def generate_next_population(self, parents, mutation_rate):
        next_population = []
        for i in range(0, len(parents), 2):
            parent1, parent2 = parents[i], parents[i+1]
            child1 = self.crossover(parent1, parent2)
            child2 = self.crossover(parent2, parent1)
            child1.mutate(mutation_rate)
            child2.mutate(mutation_rate)
            next_population.extend([child1, child2])
        self.individuals = next_population

    def get_best_individual(self):
        return min(self.individuals, key=lambda individual: individual.fitness())

### GeneticAlgorithm Class：
   - 管理基因演算法的整個流程。

In [ ]:
class GeneticAlgorithm:
    def __init__(self, pop_size=Config.POP_SIZE, generations=Config.GENERATIONS, selection_strategy=Config.SELECTION_STRATEGY,mutation_rate=Config.MUTATION_RATE):
        self.pop_size = pop_size
        self.generations = generations
        self.selection_strategy = selection_strategy
        self.mutation_rate = mutation_rate

    def run(self):
        population = Population(self.pop_size, self.selection_strategy)
        global_best_individual = population.get_best_individual()
        for generation in range(self.generations):
            parents = population.select_parents()
            population.generate_next_population(parents, self.mutation_rate)
            best_individual = population.get_best_individual()
            if best_individual.fitness() < global_best_individual.fitness():
                global_best_individual = best_individual

            print(f"Generation {generation+1}:")
            print(f"  Current Best Individual: (x: {best_individual.x:.6f}, y: {best_individual.y:.6f}), Fitness = {best_individual.fitness():.6f}")
            print(f"  Global Best Individual:  (x: {global_best_individual.x:.6f}, y: {global_best_individual.y:.6f}), Fitness = {global_best_individual.fitness():.6f}")
            print("-" * 50)

        return global_best_individual

## 主程式

In [ ]:
if __name__ == "__main__":

    if Config.SELECTION_STRATEGY == "roulette":
        selection_strategy = RouletteWheelSelection()
    elif Config.SELECTION_STRATEGY == "tournament":
        selection_strategy = TournamentSelection(tournament_size=Config.TOURNAMENT_SIZE)
    elif Config.SELECTION_STRATEGY == "rank_based":
        selection_strategy = RankBasedSelection()
    else:
        raise ValueError("Invalid selection strategy")

    ga = GeneticAlgorithm(selection_strategy=selection_strategy)
    best_solution = ga.run()
    print(f"最佳解: x = {best_solution.x}, y = {best_solution.y}, f(x, y) = {best_solution.fitness()}")